# Lambda School Data Science Module 123

## Introduction to Bayesian Inference




## Assignment - Code it up!

We used pure math to apply Bayes Theorem to drug tests. Now write Python code to reproduce the results! This is purposefully open ended - you'll have to think about how you should represent probabilities and events. You can and should look things up.

Specific goals/targets:

### 1) Write a function
`def prob_drunk_given_positive(prob_drunk_prior, false_positive_rate, true_positive_rate):`

In this example, imagine that individuals are taking a breathalyzer test with an 8% false positive rate, a 100% true positive rate, and that our prior belief about drunk driving in the population is 1/1000.
What is the probability that a person is drunk after one positive breathalyzer test?
What is the probability that a person is drunk after two positive breathalyzer tests?
How many positive breathalyzer tests are needed in order to have a probability that's greater than 95% that a person is drunk beyond the legal limit?

### 2) Explore `scipy.stats.bayes_mvs`  
Read its documentation, and experiment with it on data you've tested in other ways earlier this week.
 - Create a visualization comparing the results of a Bayesian approach to a traditional/frequentist approach. (with a large sample size they should look close to identical, however, take this opportunity to practice visualizing condfidence intervals in general. The following are some potential ways that you could visualize confidence intervals on your graph:
  - [Matplotlib Error Bars](https://matplotlib.org/3.1.1/api/_as_gen/matplotlib.pyplot.errorbar.html)
  - [Seaborn barplot with error bars](https://seaborn.pydata.org/generated/seaborn.barplot.html)
  - [Vertical ines to show bounds of confidence interval](https://www.simplypsychology.org/confidence-interval.jpg)
  - [Confidence Intervals on Box Plots](https://matplotlib.org/3.1.1/api/_as_gen/matplotlib.axes.Axes.boxplot.html)

### 3) In your own words, summarize the difference between Bayesian and Frequentist statistics

If you're unsure where to start, check out [this blog post of Bayes theorem with Python](https://dataconomy.com/2015/02/introduction-to-bayes-theorem-with-python/).



In [2]:
# TODO - code!
import numpy as np 

def prob_drunk_given_positive(prob_pos_given_drunk, prob_drunk, prob_pos):
  return (prob_pos_given_drunk*prob_drunk) / prob_pos

prior = 1 / 1000 # aka p_drunk
p_pos_given_drunk = 1
p_pos_given_not_drunk = .08
p_not_drunk = 1 - prior

# our bayes thereom here looks like: 
# P(Drunk|+) = P(+|Drunk)*P(Drunk) / (P(+|Drunk)*P(Drunk) + P(+|Not Drunk)*P(Not Drunk)

# first test probability person is drunk 
p_pos = (p_pos_given_drunk*prior) + (p_pos_given_not_drunk*p_not_drunk)
prior = prob_drunk_given_positive(p_pos_given_drunk, prior, p_pos)
prior

0.012357884330202669

In [3]:
# second test
p_not_drunk = 1-prior 
p_pos = (p_pos_given_drunk*prior) + (p_pos_given_not_drunk*p_not_drunk)
prior = prob_drunk_given_positive(p_pos_given_drunk, prior, p_pos)
prior

0.13525210993291495

In [4]:
# running till 95% confident 
test_num = 2
while prior < .95:
  test_num += 1
  p_not_drunk = 1-prior 
  p_pos = (p_pos_given_drunk*prior) + (p_pos_given_not_drunk*p_not_drunk)
  prior = prob_drunk_given_positive(p_pos_given_drunk, prior, p_pos)
  print("After test #{}, probability the person is drunk is: {}%".format(test_num, prior*100))

After test #3, probability the person is drunk is: 66.15996951348605%
After test #4, probability the person is drunk is: 96.06895076105054%


In [5]:
# 2 - scipy.stats.bayes_mvs and comparing frequentist and bayes approach
from scipy import stats 

def frequentist_interval(data, confidence=.95):
  n = len(data)
  mean = sum(data) / n 
  data = np.array(data)
  stderr = np.std(data) / np.sqrt(n) 
  t = stats.t.ppf((1+confidence)/2, n-1)
  moe = stderr * t 
  return (mean, mean-moe, mean+moe)

coinflips = np.random.binomial(n=1, p=.5, size=100)

interval = frequentist_interval(coinflips)
mean_confidence_interval, _, _ = stats.bayes_mvs(coinflips, alpha=.95)
print(interval)
mean_confidence_interval

(0.45, 0.3512864530377433, 0.5487135469622567)


Mean(statistic=0.45, minmax=(0.3507891524245659, 0.5492108475754341))

In [0]:
# 3 - frequentists vs bayesians 
'''
The key distinction between the two is the approach. Bayesians utilize prior 
knowledge as a "framework" of the liklihood of events occuring. Frequentists
utilize samples as the evidence of what occurs in the world to decipher an 
interval of where the absolute truth in all cases is. 
'''

## Resources

- [Worked example of Bayes rule calculation](https://en.wikipedia.org/wiki/Bayes'_theorem#Examples) (helpful as it fully breaks out the denominator)
- [Source code for mvsdist in scipy](https://github.com/scipy/scipy/blob/90534919e139d2a81c24bf08341734ff41a3db12/scipy/stats/morestats.py#L139)

## Stretch Goals:

- Go back and study the content from Modules 1 & 2 to make sure that you're really comfortable with them.
- Apply a Bayesian technique to a problem you previously worked (in an assignment or project work) on from a frequentist (standard) perspective
- Check out [PyMC3](https://docs.pymc.io/) (note this goes beyond hypothesis tests into modeling) - read the guides and work through some examples
- Take PyMC3 further - see if you can build something with it!